In [10]:
from functools import reduce


def remove_extra_and_merge(data: list[str]) -> list[str]:
    res = []
    for x in data:
        if x.startswith('>'):
            res.append(["", x])
        else:
            res[-1][0] += x
    return res


def parse_file(file) -> list[str]:
    with open(file, "r") as f:
        return remove_extra_and_merge([x.strip() for x in f.readlines()])


def gather_statistics(file, scaffold = False) -> str:
    data = parse_file(file)
    lens = [len(x[0]) for x in data]
    lens.sort()
    N50 = 0
    lensum = sum(lens)
    cursum = 0
    for x in lens[::-1]:
        cursum += x
        if 2 * cursum >= lensum:
            N50 = x
            break
    return "Overall number: {}, overall length: {}, max length: {}, N50: {}".format(len(data), sum(lens), max(lens), N50) \
    + (", " + gather_max_scaffold_statistics(data)) * int(scaffold)


def gather_max_scaffold_statistics(data) -> str:
    scaf = max(data, key=lambda x: len(x[0]))
    num = 0
    last = False
    for x in scaf[0]:
        if x == 'N' and not last:
            num += 1
        last = (x == 'N')
    # Надо кстати еще и записать файл longest.fasta! извините за плохой код, я в последний момент это заметил.
    with open("data/longest.fasta", "w") as f:
        print(scaf[1], scaf[0], sep="\n", file=f)
    return "scaffold with maximum length has {} gaps with the overall length of {}".format(num, scaf[0].count('N'))

In [11]:
print("Contigs:", gather_statistics("data/files_contig.fa"))
print("Scaffolds:", gather_statistics("data/files_scaffold.fa", scaffold=True))
print("Scaffolds with closed gaps: ", gather_statistics("data/files_gapClosed.fa", scaffold=True))

179307
Contigs: Overall number: 608, overall length: 3924195, max length: 179307, N50: 48054
3834161
Scaffolds: Overall number: 67, overall length: 3875155, max length: 3834161, N50: 3834161, scaffold with maximum length has 65 gaps with the overall length of 6969
3880795
Scaffolds with closed gaps:  Overall number: 67, overall length: 3922247, max length: 3880795, N50: 3880795, scaffold with maximum length has 8 gaps with the overall length of 2388


**БОНУС**

In [9]:
print("Contigs:", gather_statistics("data/small_contig.fa"))
print("Scaffolds:", gather_statistics("data/small_scaffold.fa", scaffold=True))
print("Scaffolds with closed gaps: ", gather_statistics("data/small_gapClosed.fa", scaffold=True))

Contigs: Overall number: 3444, overall length: 3916991, max length: 25762, N50: 4189
Scaffolds: Overall number: 503, overall length: 3863824, max length: 1277555, N50: 862649, scaffold with maximum length has 506 gaps with the overall length of 29410
Scaffolds with closed gaps:  Overall number: 503, overall length: 3853327, max length: 1275044, N50: 859170, scaffold with maximum length has 37 gaps with the overall length of 13917


Видим: при сокращении размера семпла в 10 раз контигов и скаффолдов стало больше, но их длины уменьшились; количество и суммарная длина гэпов в максимальном скаффолде увеличились.